![University of Tehran](./img/UT.png)
#   <font color='red'><center>AI CA 1<center></font> 
## <center>Dr. Fadaei<center>
### <center>Daniyal Maroufi<center>
### <center>810098039<center>

# Aim

This assignment aims to solve a problem using Informed and Uninformed search algorithms such as BTS, IDS, and A* algorithm. Finally, we will compare the time complexity of these algorithms.

# Problem Definition

## Intial State

The initial state is the state that our agent will start from. In this case, Gandalf starts from his state at a given position in the input.

## Action

The agent can do an action to reach another state. In this case, Gandolf can have six actions: going up, right, down, and left and picking a friend, and placing them in another position.

## Transition Model

UP action: If it is permitted, the agent will move from state (x,y) to state (x-1,y)

DOWN action: If it is permitted, the agent will move from state (x,y) to state (x+1,y)

RIGHT action: If it is permitted, the agent will move from state (x,y) to state (x,y+1)

LEFT action: If it is permitted, the agent will move from state (x,y) to state (a,y-1)

PICK action: the agent will pick the friend

PLACE action: the agent will place the friend

## Goal State

The goal state is that the agent has successfully done its tasks. In this case, Gandalf must place all the friends in their goal positions and reach the final point (Gandor).

# Algorithms

## BFS

The Breadth-First Search (BFS) is an algorithm for traversing or searching tree or graph data structures. It explores all the nodes at the present depth before moving to the next depth level. BFS is complete, and it will give us an optimal solution with time complexity of $O(b^d)$.

## IDS

The IDS or Iterative Deepening Search is an algorithm based on DFS which controls the depth before expanding another node. IDS leads to an optimal solution. Unlike DFS, its time complexity is $O(bm)$.

## A*

A* algorithm searches for the shortest path between the initial and the final state. It is used in various applications, such as maps.
In maps, the A* algorithm calculates the shortest distance between the source (initial state) and the destination (final state).
$g(n)$ is the cost of reaching the current state, and $h(n)$ is the heuristic function that estimates the cost of getting to the final state from the current node $n$.
A* search is complete, and its time complexity is $O(b^d)$.

# Classes

In [1]:
import queue
from time import time
import copy

## Node

In [174]:
class Node(object):
    def __init__(self,position):
        self.state=tuple(position)
        self.parent=None
        self.path=''
        self.picked_RF=None
        self.placed_RFs=set()
        self.inOrk=None
        self.presence=0

    def __eq__(self, other):
        if self.state==other.state and self.path==other.path and self.picked_RF==other.picked_RF and self.placed_RFs==other.placed_RFs:
            return True
        return False

    def __hash__(self):
        return hash((self.state,str(self.placed_RFs),self.picked_RF))
    


In [175]:
def createChild(parent=Node((-1,-1))):
    child=Node(parent.state)
    child.parent=parent
    child.path=copy.deepcopy(parent.path)
    child.picked_RF=parent.picked_RF
    child.placed_RFs=copy.deepcopy(parent.placed_RFs)
    child.inOrk=parent.inOrk
    child.presence=parent.presence
    return child

## Problem

In [268]:
class Problem():
    def __init__(self, test_name):
        self.orks=[]
        self.RFs_initial_pos=[]
        self.RFs_goal_pos=[]
        with open('./tests/'+test_name+'.txt') as f:
            self.n,self.m = tuple(map(int,f.readline().split()))
            self.gandalf_initial_pos = tuple(map(int,f.readline().split()))
            self.gandor = tuple(map(int,f.readline().split()))
            self.k,self.l = tuple(map(int,f.readline().split()))
            for _ in range(self.k):
                x,y,c = tuple(map(int,f.readline().split()))
                self.orks.append((x,y,c))
            for _ in range(self.l):
                x,y = tuple(map(int,f.readline().split()))
                self.RFs_initial_pos.append((x,y))
            for _ in range(self.l):
                x,y = tuple(map(int,f.readline().split()))
                self.RFs_goal_pos.append((x,y))
        self.allRFs=dict(zip(self.RFs_initial_pos,self.RFs_goal_pos))

        self.orks_map=[[0]*self.m for _ in range(self.n)]
        for ork,(x,y,c) in enumerate(self.orks):
            for i in range(-c,c+1):
                for j in range(-c,c+1):
                    if abs(i)+abs(j)<=c and x+i>=0 and x+i<self.n and y+j>=0 and y+j<self.m:
                            self.orks_map[x+i][y+j]=ork+1
    
    def getInitialNode(self):
        node=Node(self.gandalf_initial_pos)
        return node
    
    def goalTest(self, node=Node((-1,-1))):
        print('goaltestcalled')
        if len(node.placed_RFs)==self.l and node.state==self.gandor:
            return True
        return False

    def canGo(self,node=Node((-1,-1))):
        print('cangocalled')
        x,y=node.state
        if x<0 or x>=self.n or y<0 or y>=self.m:
            return False
        if self.k>0:
            if self.orks_map[x][y]==node.inOrk:
                node.presence+=1
                if node.presence>self.orks[node.inOrk-1]:
                    return False
        return True
    
    def actions(self):
        print('actionscalled')
        return {'pick':'pick','place':'place','R':(0,1),'D':(1,0),'U':(-1,0),'L':(0,-1)}



# BFS

In [269]:
def bfs(problem):
    frontier=queue.Queue()
    explored=set() # saves the hash of the states in the frontier
    # exploredSet=set()
    seen_states = 0
    root=problem.getInitialNode()
    frontier.put(root)
    explored.add(hash(root))
    if problem.goalTest(root):
        return 1,root.path
    while not frontier.empty():
        # print('frontier:',[a for a in explored])
        node=frontier.get()
        # print(hash(node))
        # print(node.path)
        # print(node.state)
        for action,transition in problem.actions().items():
            # print('action:',action)
            if action == 'pick':
                # print('path',node.path)
                # print('pick?','picked:',node.picked_RF,'state:',node.state,'initiallist:',problem.RFs_initial_pos)
                if node.picked_RF is None and node.state in problem.RFs_initial_pos and problem.allRFs[node.state] not in node.placed_RFs:
                    
                    child_with_RF=createChild(node)
                    child_with_RF.picked_RF=problem.allRFs[node.state]

                    # child_with_RF=Node(node.state)
                    # child_with_RF.parent=node
                    # child_with_RF.path=copy.deepcopy(node.path)
                    # child_with_RF.picked_RF=problem.allRFs[node.state]
                    # child_with_RF.placed_RFs=copy.deepcopy(node.placed_RFs)
                    # child_with_RF.inOrk=node.inOrk
                    # child_with_RF.presence=node.presence

                    # print('picked!!')
                    seen_states+=1
                    child_hash=hash(child_with_RF)
                    # if child_hash not in explored:
                    frontier.put(child_with_RF)
                    explored.add(child_hash)
            elif action == 'place':
                if node.state == node.picked_RF:
                    # print('placed!!')

                    child=createChild(node)
                    child.placed_RFs.add(node.state)
                    child.picked_RF=None

                    # child=Node(node.state)
                    # child.parent=node
                    # child.path=copy.deepcopy(node.path)
                    # child.picked_RF=None
                    # child.placed_RFs=copy.deepcopy(node.placed_RFs)
                    # child.placed_RFs.add(node.state)
                    # child.inOrk=node.inOrk
                    # child.presence=node.presence

                    seen_states+=1
                    child_hash=hash(child)
                    # if child_hash not in explored:
                    frontier.put(child)
                    explored.add(hash(child))
                    # print('placed!:):')
                    # print(child.placed_RFs)
            else:
                # print('node current state:',node.state)
                # print('trans:',transition)

                child=Node(node.state)
                


                # child=createChild(node)


                child.state=(node.state[0]+transition[0],node.state[1]+transition[1])

                # print('child moved state:',child.state)

                # print('canGo?',problem.canGo(child))

                if problem.canGo(child):
                    # print('couldGo:',child.state)
                    child.parent=node
                    child.path=copy.deepcopy(node.path)+action
                    child.picked_RF=node.picked_RF
                    child.placed_RFs=copy.deepcopy(node.placed_RFs)
                    child.inOrk=node.inOrk
                    child.presence=node.presence
                    # child.path+=action
                    if hash(child) not in explored:
                        seen_states+=1
                        if problem.goalTest(child):
                            return seen_states, child.path
                        frontier.put(child)
                        explored.add(hash(child))
                        # print('added to front',child.path)
    return 0,''





In [270]:
test0=Problem('test_00')

In [271]:
tic=time()
a,b=bfs(test0)
toc=time()
print(a,b,len(b))
print('time',(toc-tic)*1000)

goaltestcalled
actionscalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
actionscalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
cangocalled
actionscalled
cangocalled
cangocalled
goaltestcalled
cangocalled
cangocalled
goaltestcalled
actionscalled
cangocalled
cangocalled
cangocalled
cangocalled
goaltestcalled
actionscalled
cangocalled
cangocalled
cangocalled
cangocalled
actionscalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
cangocalled
actionscalled
cangocalled
cangocalled
goaltestcalled
cangocalled
cangocalled
actionscalled
cangocalled
cangocalled
cangocalled
cangocalled
actionscalled
cangocalled
cangocalled
cangocalled
cangocalled
goaltestcalled
actionscalled
cangocalled
cangocalled
cangocalled
cangocalled
actionscalled
cangocalled
cangocalled
cangocalled
cangocalled
actionscalled
cangocalled
goaltestcalled
cangocalled
goaltestcalled
cangocalle

In [222]:
for _ in range(3):
    tic=time()
    a,b=bfs(test0)
    toc=time()
    print(a,b,len(b))
    print('time',(toc-tic)*1000)

449 RRRRRRRDDDDDLLLLLLLRDDRRRRRUURDD 32
time 98.3438491821289
449 RRRRRRRDDDDDLLLLLLLRDDRRRRRUURDD 32
time 61.30242347717285
449 RRRRRRRDDDDDLLLLLLLRDDRRRRRUURDD 32
time 62.23726272583008


In [128]:
for _ in range(3):
    tic=time()
    a,b=bfs(test0)
    toc=time()
    print(a,b,len(b))
    print('time',(toc-tic)*1000)

10474 RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR 48
time 2330.172061920166
10474 RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR 48
time 2597.503662109375
10474 RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR 48
time 2205.070972442627


In [35]:
len('RRRRRRRLLLLDDDDLDLLRDDRUURURRRDRDD')

34

In [50]:
len('RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR')

48

In [39]:
test0.RFs_goal_pos


[(5, 0), (5, 6)]

cc False
bb: True


In [ ]:
aa=Node((0,0))
a=Node((1,0))
a.parent=aa
b=Node((2,0))
b.parent=a
c=Node((3,0))
c.parent=b
d=Node((4,0))
d.parent=c

print('aa:',test0.countPresence(test0.orks[0],d))

print('cc',test0.isInRegionOfOrk(test0.orks[0],d))
print('bb:',test0.canGo(d))



# a.placed_RFs.add((2,2))
# b.placed_RFs=a.placed_RFs
# b.placed_RFs.add((2,3))
# b.placed_RFs

In [82]:
aa=Node((0,0))
a=Node((0,1))
a.parent=aa
b=Node((0,2))
b.parent=a
c=Node((0,3))
c.parent=b
d=Node((0,4))
d.parent=c

print('aa:',test0.countPresence(test0.orks[0],c))

print('cc',test0.isInRegionOfOrk(test0.orks[0],c))
print('bb:',test0.canGo(c))


aa: 1
cc True
bb: False
